# Darstellung der automatisierten Fälle

In [1]:
import numpy as np
import xarray as xr
from standard_config import *
import glob
import pandas as pd

from plotting_tools.colormaps import radar_cmap as rc

from io_tools import radolan
import load_satellite_data as lsd
import read_linet_data as rld
import datetime as dt
import MSGtools as mst
from analysis_tools import grid_and_interpolation as gi

import matplotlib.pyplot as plt
%matplotlib inline

import radar_cmap as rc
rcmap,rnorm = rc.radar_cmap()

import haci_filters as hf

from PIL import Image

/vols/talos/local/anaconda2-5.0.0/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def normalise2range(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def array_to_256(array):
    return (array*255.999).astype("uint8")

def day_natural_composite(vis006_data,vis008_data,nir016_data,factor=1,gamma=1):
    blue = array_to_256(np.clip(vis006_data/factor,0,1)**(1./gamma))
    green = array_to_256(np.clip(vis008_data/factor,0,1)**(1./gamma))
    red = array_to_256(np.clip(nir016_data/factor,0,1)**(1./gamma))
    
    return np.dstack([red,green,blue]).astype("uint8")

def scale_array_min_max(array_data,range_min=0,range_max=1):
    """
    Scales a array into the chosen range.
    
    Inputs:
    -------
    array_data: numpy array of floats or integers, 2d
        array to scale
    range_min: int or float, default = 0
        minimum value of the range to scale array to,
    range_max: int or float, default = 1
        maximum value of the range to scale array to,
        
    Returns:
    --------
    scaled_array: numpy array of floats, 2d
    """
    # get array extrema
    array_min = np.min(array_data)
    array_max = np.max(array_data)

    # derive conversion parameters
    a = (range_max - range_min) / (array_max - array_min)
    b = range_max - a * array_max

    # scale array
    scaled_array = a * array_data + b
    
    return scaled_array

def add_hrv_texture2nc(nc,hrv):
    nc_lab = color.rgb2lab(nc)
    l_min = nc_lab[...,0].min()
    l_max = nc_lab[...,0].max()
    
    l_hrv_scaled = nc_lab[...,0] * hrv
    l_hrv_scaled = scale_array_min_max(l_hrv_scaled,l_min,l_max)

    nc_lab[...,0] = l_hrv_scaled
    
    return color.lab2rgb(nc_lab)

def convert_haci_bbox2msg(object_data,box_size=35):
    from io_tools import radolan
    from analysis_tools import grid_and_interpolation as gi
    
    r = radolan.Radolan()
    slon,slat = mst.get_msg_lon_lat('eu')
    
    centroid_line = object_data.l00 + (object_data.l01 - object_data.l00) / 2.
    centroid_column = object_data.c00 + (object_data.c01 - object_data.c00) / 2.
    
    centroid_lon = r.lon[int(np.rint(centroid_line)),int(np.rint(centroid_column))]
    centroid_lat = r.lat[int(np.rint(centroid_line)),int(np.rint(centroid_column))]
    
    box_lon_min = r.lon[int(np.rint(object_data.l0)),
                        int(np.rint(object_data.c0))]
    
    box_lat_min = r.lat[int(np.rint(object_data.l0)),
                        int(np.rint(object_data.c0))]
    
    box_lon_max = r.lon[int(np.rint(object_data.l1)),
                        int(np.rint(object_data.c1))]
    
    box_lat_max = r.lat[int(np.rint(object_data.l1)),
                        int(np.rint(object_data.c1))]
    
    sat_row, sat_col = gi.get_index((centroid_lon,centroid_lat),slon,slat)
    sat_row_min = sat_row - (box_size // 2)
    sat_row_max = sat_row + (box_size // 2) + 1
    sat_col_min = sat_col - (box_size // 2)
    sat_col_max = sat_col + (box_size // 2) + 1
    
    sat_lon_min = slon[sat_row_min,sat_col_min]
    sat_lat_min = slat[sat_row_max,sat_col_min]
    sat_lon_max = slon[sat_row_max,sat_col_max]
    sat_lat_max = slat[sat_row_min,sat_col_max]
    
    out_dict = {'lon_min':sat_lon_min,
                'lat_min':sat_lat_min,
                'lon_max':sat_lon_max,
                'lat_max':sat_lon_max,
                'row_min':sat_row_min,
                'row_max':sat_row_max,
                'col_min':sat_col_min,
                'col_max':sat_col_max}
    
    return out_dict    

Wir laden uns den vorgefilterten Datensatz und erzeugen NC-Kompositdarstellungen des CI-Zeitschrittes.

In [3]:
#haci_cases= pd.read_csv("{}/2019-01_trackingstudie/data/haci_cases_2013_filtered.csv".format(proj_path))
haci_cases= pd.read_csv("/vols/satellit/home/lenk/data/HACI_bbox_data/haci_objects_filtered.csv")

In [4]:
haci_cases.head()

,date,time,id,flags,t0,t1,l00,l01,c00,c01,l0,l1,c0,c1,t_duration,area0,area_max,darea
0,2008-01-07,09:45,1370,0,117,123,307,308,705,706,295,337,656,725,6,1,2898,2897
1,2008-01-17,17:30,969,0,210,216,670,673,269,273,670,759,261,304,6,12,3827,3815
2,2008-01-22,05:55,672,0,71,77,281,283,706,708,257,310,659,726,6,4,3551,3547
3,2008-02-06,03:40,734,0,44,50,282,283,706,707,263,345,672,735,6,1,5166,5165
4,2008-02-06,13:05,2457,0,157,163,299,300,729,730,294,308,705,741,6,1,504,503


Als nächstes erzeugen wir uns Ausschnitte rund um die Punkte.

In [6]:
channels = ["VIS006","VIS008","IR_016",'HRV']

In [7]:
nc_path = "{}/2019-01_trackingstudie/pics/haci_auto_nc/".format(proj_path)

In [ ]:
cloudiness = dict()

for i, case in haci_cases.iterrows():
    try:
        case_id = "{}_{}".format(case.date.replace("-",""),case.id)
        -
        ncs = []

        for i in xrange(len(cutouts['HRV'])):
            ncs.append(day_natural_composite(cutouts['VIS006'][i],cutouts['VIS008'][i],cutouts['IR_016'][i],1,1))

        ncs = np.hstack(ncs)
        nc_image = Image.fromarray(ncs)
        nc_image.save("{}/{}.png".format(nc_path,case_id))
        
        cloudy = len(cutouts['HRV'][1][np.where(cutouts['HRV'][1]>=0.3)])
        clear = len(cutouts['HRV'][1][np.where(cutouts['HRV'][1]<0.3)])

        cloudiness[case_id] = cloudy / float(cloudy + clear)   
    except:
        continue

Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/01/07/msg?-sevi-20080107t0915z-l15hdf-rss-eu.c2.h5  does not exist!
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hrit/2008/01/07/msg?-sevi-20080107t0915z-l15hrit-rss.c2.tar  does not exist!
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/01/17/msg?-sevi-20080117t1700z-l15hdf-rss-eu.c2.h5  does not exist!
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hrit/2008/01/17/msg?-sevi-20080117t1700z-l15hrit-rss.c2.tar  does not exist!
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/01/22/msg?-sevi-20080122t0525z-l15hdf-rss-eu.c2.h5  does not exist!
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hrit/2008/01/22/msg?-sevi-20080122t0525z-l15hrit-rss.c2.tar  does not exist!
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/02/06/msg?-sevi-20080206t

Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/05/01/msg?-sevi-20080501t1555z-l15hdf-rss-eu.c2.h5  does not exist!
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hrit/2008/05/01/msg?-sevi-20080501t1555z-l15hrit-rss.c2.tar  does not exist!
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/05/02/msg?-sevi-20080502t1430z-l15hdf-rss-eu.c2.h5  does not exist!
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hrit/2008/05/02/msg?-sevi-20080502t1430z-l15hrit-rss.c2.tar  does not exist!
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/05/03/msg?-sevi-20080503t1345z-l15hdf-rss-eu.c2.h5  does not exist!
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hrit/2008/05/03/msg?-sevi-20080503t1345z-l15hrit-rss.c2.tar  does not exist!
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/05/11/msg?-sevi-20080511t

Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/05/15/msg?-sevi-20080515t1435z-l15hdf-rss-eu.c2.h5  does not exist!
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hrit/2008/05/15/msg?-sevi-20080515t1435z-l15hrit-rss.c2.tar  does not exist!
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/05/15/msg?-sevi-20080515t1600z-l15hdf-rss-eu.c2.h5  does not exist!
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hrit/2008/05/15/msg?-sevi-20080515t1600z-l15hrit-rss.c2.tar  does not exist!
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/05/15/msg?-sevi-20080515t1755z-l15hdf-rss-eu.c2.h5  does not exist!
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hrit/2008/05/15/msg?-sevi-20080515t1755z-l15hrit-rss.c2.tar  does not exist!
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/05/16/msg?-sevi-20080516t

... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-VIS008___-000007___-200805261650-__
... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-VIS008___-000008___-200805261650-__
... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-HRV______-000020___-200805261650-__
... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-HRV______-000021___-200805261650-__
... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-HRV______-000022___-200805261650-__
... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-HRV______-000023___-200805261650-__
... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-VIS006___-000007___-200805261650-__
... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-VIS006___-000008___-200805261650-__
... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-IR_016___-000007___-200805261650-__
... reading  /tmp/hrit2043106481/H-000-MSG1__-MSG1_RSS____-IR_016___-000008___-200805261650-__

Combine segments

Do calibration
Region suggests 

... reading  /tmp/hrit3644660345/H-000-MSG1__-MSG1_RSS____-HRV______-000021___-200805281500-__
... reading  /tmp/hrit3644660345/H-000-MSG1__-MSG1_RSS____-HRV______-000022___-200805281500-__
... reading  /tmp/hrit3644660345/H-000-MSG1__-MSG1_RSS____-HRV______-000023___-200805281500-__
... reading  /tmp/hrit3644660345/H-000-MSG1__-MSG1_RSS____-VIS006___-000007___-200805281500-__
... reading  /tmp/hrit3644660345/H-000-MSG1__-MSG1_RSS____-VIS006___-000008___-200805281500-__
... reading  /tmp/hrit3644660345/H-000-MSG1__-MSG1_RSS____-IR_016___-000007___-200805281500-__
... reading  /tmp/hrit3644660345/H-000-MSG1__-MSG1_RSS____-IR_016___-000008___-200805281500-__

Combine segments

Do calibration
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2008/05/28/msg?-sevi-20080528t1530z-l15hdf-rss-eu.c2.h5  does not exist!
... reading  /tmp/hrit2837757616/H-000-MSG1__-MSG1_RSS____-VIS008___-000007___-200805281530-__
... reading  /tmp/hrit2837757616/H-000-MSG

...

... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-VIS008___-000007___-200905210910-__
... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-VIS008___-000008___-200905210910-__
... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-HRV______-000020___-200905210910-__
... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-HRV______-000021___-200905210910-__
... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-HRV______-000022___-200905210910-__
... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-HRV______-000023___-200905210910-__
... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-VIS006___-000007___-200905210910-__
... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-VIS006___-000008___-200905210910-__
... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-IR_016___-000007___-200905210910-__
... reading  /tmp/hrit1105555696/H-000-MSG1__-MSG1_RSS____-IR_016___-000008___-200905210910-__

Combine segments

Do calibration
Region suggests 

... reading  /tmp/hrit9136338584/H-000-MSG1__-MSG1_RSS____-HRV______-000020___-200905211230-__
... reading  /tmp/hrit9136338584/H-000-MSG1__-MSG1_RSS____-HRV______-000021___-200905211230-__
... reading  /tmp/hrit9136338584/H-000-MSG1__-MSG1_RSS____-HRV______-000022___-200905211230-__
... reading  /tmp/hrit9136338584/H-000-MSG1__-MSG1_RSS____-HRV______-000023___-200905211230-__
... reading  /tmp/hrit9136338584/H-000-MSG1__-MSG1_RSS____-VIS006___-000007___-200905211230-__
... reading  /tmp/hrit9136338584/H-000-MSG1__-MSG1_RSS____-VIS006___-000008___-200905211230-__
... reading  /tmp/hrit9136338584/H-000-MSG1__-MSG1_RSS____-IR_016___-000007___-200905211230-__
... reading  /tmp/hrit9136338584/H-000-MSG1__-MSG1_RSS____-IR_016___-000008___-200905211230-__

Combine segments

Do calibration
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2009/05/21/msg?-sevi-20090521t1300z-l15hdf-rss-eu.c2.h5  does not exist!
... reading  /tmp/hrit1156159653/H-000-MSG

... reading  /tmp/hrit1413833465/H-000-MSG1__-MSG1_RSS____-HRV______-000022___-200905211425-__
... reading  /tmp/hrit1413833465/H-000-MSG1__-MSG1_RSS____-HRV______-000023___-200905211425-__
... reading  /tmp/hrit1413833465/H-000-MSG1__-MSG1_RSS____-VIS006___-000007___-200905211425-__
... reading  /tmp/hrit1413833465/H-000-MSG1__-MSG1_RSS____-VIS006___-000008___-200905211425-__
... reading  /tmp/hrit1413833465/H-000-MSG1__-MSG1_RSS____-IR_016___-000007___-200905211425-__
... reading  /tmp/hrit1413833465/H-000-MSG1__-MSG1_RSS____-IR_016___-000008___-200905211425-__

Combine segments

Do calibration
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2009/05/21/msg?-sevi-20090521t1455z-l15hdf-rss-eu.c2.h5  does not exist!
... reading  /tmp/hrit1707410534/H-000-MSG1__-MSG1_RSS____-VIS008___-000007___-200905211455-__
... reading  /tmp/hrit1707410534/H-000-MSG1__-MSG1_RSS____-VIS008___-000008___-200905211455-__
... reading  /tmp/hrit1707410534/H-000-MSG

... reading  /tmp/hrit2721138604/H-000-MSG1__-MSG1_RSS____-VIS006___-000007___-200905211510-__
... reading  /tmp/hrit2721138604/H-000-MSG1__-MSG1_RSS____-VIS006___-000008___-200905211510-__
... reading  /tmp/hrit2721138604/H-000-MSG1__-MSG1_RSS____-IR_016___-000007___-200905211510-__
... reading  /tmp/hrit2721138604/H-000-MSG1__-MSG1_RSS____-IR_016___-000008___-200905211510-__

Combine segments

Do calibration
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2009/05/21/msg?-sevi-20090521t1410z-l15hdf-rss-eu.c2.h5  does not exist!
... reading  /tmp/hrit8099918205/H-000-MSG1__-MSG1_RSS____-VIS008___-000007___-200905211410-__
... reading  /tmp/hrit8099918205/H-000-MSG1__-MSG1_RSS____-VIS008___-000008___-200905211410-__
... reading  /tmp/hrit8099918205/H-000-MSG1__-MSG1_RSS____-HRV______-000020___-200905211410-__
... reading  /tmp/hrit8099918205/H-000-MSG1__-MSG1_RSS____-HRV______-000021___-200905211410-__
... reading  /tmp/hrit8099918205/H-000-MSG

... reading  /tmp/hrit8703110832/H-000-MSG1__-MSG1_RSS____-IR_016___-000008___-200905211450-__

Combine segments

Do calibration
Region suggests use of hdf file
ERROR:  /vols/altair/datasets/eumcst//msevi_rss/l15_hdf/eu/2009/05/21/msg?-sevi-20090521t1520z-l15hdf-rss-eu.c2.h5  does not exist!
... reading  /tmp/hrit2512646825/H-000-MSG1__-MSG1_RSS____-VIS008___-000007___-200905211520-__
... reading  /tmp/hrit2512646825/H-000-MSG1__-MSG1_RSS____-VIS008___-000008___-200905211520-__
... reading  /tmp/hrit2512646825/H-000-MSG1__-MSG1_RSS____-HRV______-000020___-200905211520-__
... reading  /tmp/hrit2512646825/H-000-MSG1__-MSG1_RSS____-HRV______-000021___-200905211520-__
... reading  /tmp/hrit2512646825/H-000-MSG1__-MSG1_RSS____-HRV______-000022___-200905211520-__
... reading  /tmp/hrit2512646825/H-000-MSG1__-MSG1_RSS____-HRV______-000023___-200905211520-__
... reading  /tmp/hrit2512646825/H-000-MSG1__-MSG1_RSS____-VIS006___-000007___-200905211520-__
... reading  /tmp/hrit2512646825/H-000-MSG

In [13]:
cloudiness = pd.DataFrame(cloudiness)

ValueError: If using all scalar values, you must pass an index

In [14]:
cloudiness

{73: 0.0049382716049382715,
 116: 0.037530864197530864,
 126: 1.0,
 130: 0.028641975308641977,
 134: 0.26024691358024693,
 136: 0.2750617283950617,
 139: 0.20839506172839506,
 140: 0.4997530864197531,
 149: 0.5604938271604938,
 157: 0.0,
 159: 0.4019753086419753,
 160: 0.3506172839506173,
 163: 0.11308641975308642,
 166: 0.044444444444444446,
 171: 0.6212345679012345,
 172: 0.1511111111111111,
 178: 0.0345679012345679,
 184: 0.14271604938271604,
 187: 0.4271604938271605,
 188: 0.988641975308642,
 189: 0.4414814814814815,
 190: 0.2730864197530864,
 191: 0.21481481481481482,
 193: 0.291358024691358,
 194: 0.4745679012345679,
 195: 1.0,
 196: 1.0,
 197: 0.17925925925925926,
 198: 0.38123456790123456,
 199: 0.5185185185185185,
 201: 0.8024691358024691,
 202: 0.2696296296296296,
 203: 0.01580246913580247,
 207: 0.44296296296296295,
 208: 0.4222222222222222,
 210: 0.16246913580246913,
 211: 0.7338271604938271,
 216: 0.16197530864197532,
 218: 0.9980246913580247,
 222: 0.5027160493827161,
 22